In [1]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [2]:
DIR="..\\..\\chunk\\18\\"
checks = pd.read_csv(f"{DIR}data.csv")
checks.drop(['name'], axis=1, inplace=True)

with open(f"{DIR}Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_14100\191298464.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv(f"{DIR}data.csv")


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [4]:
#какие товары обращаются в продаже
list(product_dict.items())[:5]
#первое и второе числа в выборке из словаря:
# product_id -ключ аптечной позиции в словаре
# contact_id -касса, через которую прошла данная товарная позиция


[('168308', '(197312) Пакет-майка 25см х 45см 906'),
 ('134832', '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404'),
 ('101384', '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('168570', '(197309) Пакет 28см х 50см 906'),
 ('146960', '(111023) Пакет "Аптека Озерки" 28 х 35см 906')]

In [5]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [6]:
checks.dropna(inplace=True)
checks[checks['sale_date_date'].str.startswith("20")!=True][:4]


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col


In [7]:
checks.shape

(20000000, 9)

In [8]:
# checks = checks[checks['sale_date_date'].str.startswith("20")] #ненужная фильтрация

checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [12]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")
from tqdm import tqdm
tmpcnt=0
lastpos=0

def preprocess_txt(line):
    global tmpcnt
    global lastpos
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    tmpcnt+=1
    t.update((tmpcnt-lastpos,lastpos:=tmpcnt)[0])
    return spls

t = tqdm(total=len(product_dict.keys())) # инициализировать прогресбар размером с длину словаря
sentences = [preprocess_txt(k)[1:] for _,k in product_dict.items()] #исключить ненужный по смыслу лекарства номер кассы, стоящий в позиции 0
t.close()

100%|██████████| 30418/30418 [00:54<00:00, 561.29it/s]


In [14]:
# после обработки товарные позиции выглядят так:
sentences[:5]

[['пакетмайк', '25см', 'х', '45см', '906'],
 ['перекись', 'водород', 'рр', 'наружн', '3', 'флполимерна', '100мл', '404'],
 ['салициловый', 'кислота', 'рр', 'спирт', '2', 'фл', '40мл', 'n1', '404'],
 ['пакет', '28см', 'х', '50см', '906'],
 ['пакет', 'аптека', 'озерко', '28', 'х', '35см', '906']]

In [16]:
# проверка  на соответствие готовых преобразованных позиций и оригинальных товарных позиций из словаря
counter=0
for key,name in product_dict.items():
    print(f'{sentences[counter]}|{key}:{name}')
    counter+=1
    if counter>15:break


['пакетмайк', '25см', 'х', '45см', '906']|168308:(197312) Пакет-майка 25см х 45см 906
['перекись', 'водород', 'рр', 'наружн', '3', 'флполимерна', '100мл', '404']|134832:(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404
['салициловый', 'кислота', 'рр', 'спирт', '2', 'фл', '40мл', 'n1', '404']|101384:(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404
['пакет', '28см', 'х', '50см', '906']|168570:(197309) Пакет 28см х 50см 906
['пакет', 'аптека', 'озерко', '28', 'х', '35см', '906']|146960:(111023) Пакет "Аптека Озерки" 28 х 35см 906
['ранитидина', 'тб', '150мг', 'уп', 'n20', '595']|128700:(51806) Ранитидин тб 150мг уп N20 595
['платок', 'носовой', 'зев', 'кидс', '№10', '721']|52539:(19416) Платки носовые "Зева Кидс" №10 721
['пакет', 'майк', 'благодарить', 'покупка', '2814', 'х', '43см', '№1', '1']|94270:(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1
['игла', 'микрофайн', 'дшприцручка', '30g', '038мма', 'n100', '759']|77426:(5605) Иглы Микро-Файн д/шпри

In [20]:

# Подготовим кандидатогенератор, который будет выдавать лекарства, похожие по текстовому описанию на те, которые выбрал покупатель
# Обучим Fasttext и заэмбедим лекарства
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')


In [21]:

# Будем хранить соответствия не только product_id-> лекарство, но и лекарство-> product_id, чтобы быстрее находить эмбеддинги айтемов
index_map = {}
reverse_index_map = {}
counter = 0
import time
t = tqdm(total=len(product_dict.keys())) # инициализировать прогресбар размером с длину словаря
tmpcnt=0
lastpos=0
start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара

# каждую аптечную позицию внести в индекс для последующей выборки похожих на заданную позиций
for key,name in product_dict.items():
    tmpcnt+=1
    if time.time()-start_time>2:
        start_time = time.time()
        t.update((tmpcnt-lastpos,lastpos:=tmpcnt)[0])
    n_ft = 0
    index_map[counter]=key
    reverse_index_map[key] = counter
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in sentences[counter]: #каждая позиция в словаре product_dict соответствует позиции в обработанных наборах слов sentences
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft #простейший способ хранить вектор названия из нескольких слов, как среднее векторов составляющих слов
    ft_index_names.add_item(counter, vector_ft)
    counter += 1
t.update((tmpcnt-lastpos,lastpos:=tmpcnt)[0])
t.close()
# 
ft_index_names.build(10)

100%|██████████| 30418/30418 [00:02<00:00, 14154.29it/s]


True

In [17]:
qq={'1':11,'2':22}
print('1' in qq)
print(11 in qq)

True
False


In [24]:
# reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list): #на вход список product_id
    current_vector = np.zeros(20) #вектор- для набора усредненной характеристики товарной группы из списка items_list
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        # reverse_index_map в качестве ключей содержит product_id, а в качестве элементов - ключи для index_map, на основе которых построен annoy
        if iid not in reverse_index_map: #если попалась позиция, которая предварительно не попала в индекс, то не учитывать ее вектор
            c -= 1
            continue
        # извлечь ранее помещенные в annoy векторы для известных товарных позиций, 
        current_vector += np.array(ft_index_names.get_item_vector(reverse_index_map[iid]))
    # взять для суммы их векторов среднее и выдать похожие позиции с помощью алгоритма похожести annoy
    return [(index_map[i],product_dict[index_map[i]]) for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [29]:
# найти похожие для позиций
[product_dict[i] for i in ['134832', '101384']]

['(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404']

In [30]:
recommend(['101384'])

[('101384', '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('135553', '(3348) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('37381', '(109185) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('111281', '(10510) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('133946', '(64588) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('117847', '(115230) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('75961', '(16471) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('62299', '(13585) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('89629', '(15095) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('141272', '(15965) Салициловая кислота р-р спирт 2% фл 40мл N1 404')]

In [33]:
recommend(['101384'])[:][:]

[('101384', '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('135553', '(3348) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('37381', '(109185) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('111281', '(10510) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('133946', '(64588) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('117847', '(115230) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('75961', '(16471) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('62299', '(13585) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('89629', '(15095) Салициловая кислота р-р спирт 2% фл 40мл N1 404'),
 ('141272', '(15965) Салициловая кислота р-р спирт 2% фл 40мл N1 404')]

In [25]:
recommend(['134832', '101384'])

[('151683', '(15182) Перекись водорода р-р наружн. 3% фл. 40мл 404'),
 ('85149', '(76155) Перекись водорода р-р наружн. 3% фл. 40мл 404'),
 ('153088', '(120843) Перекись водорода р-р наружн. 3% фл. 40мл 404'),
 ('132006', '(104259) Перекись водорода р-р наружн. 3% фл. 100мл 404'),
 ('151336', '(101739) Чемеричная вода р-р фл 100мл N1 661'),
 ('105177', '(114421) Чемеричная вода р-р фл 100мл N1 661'),
 ('95282', '(38983) Чемеричная вода р-р фл 100мл N1 661'),
 ('79926', '(40433) Чемеричная вода р-р фл 100мл N1 661'),
 ('131852', '(34748) Аммифурин р-р 0,3% фл 50мл N1 467'),
 ('149587', '(74654) Камфорный спирт р-р 2% фл 40мл 648')]

In [27]:
product_dict['151683']

'(15182) Перекись водорода р-р наружн. 3% фл. 40мл 404'

In [28]:
product_dict['85149']

'(76155) Перекись водорода р-р наружн. 3% фл. 40мл 404'

In [ ]:
# в словаре товарных позиций присутствуют одинаковые с разными product_id

In [10]:
# Обучение эмбеддингов по последовательности
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

In [11]:
# Сформируем последовательности

from tqdm import tqdm 
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1066363157.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5457574 [00:00<?, ?it/s]

In [13]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [14]:
sentences[0]

['115617', '142296', '37070']

In [16]:
modelW2V.wv.similar_by_vector

<bound method KeyedVectors.similar_by_vector of <gensim.models.keyedvectors.KeyedVectors object at 0x000001E477776160>>

In [19]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

In [20]:
recommend_w2v(['99821', '138583', '45321', '134475'])

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_2832\1051936176.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]


['(181271) Зубная паста Колгейт Древние секреты Безупречная Свежесть 75мл 529',
 '(67162) Габагамма капс. 300мг №50 641',
 '(187610) Гиалрипайер-10 Хондрорепарант гель гиалуроновый д/инъек.фл.0,8% 5мл №1 646',
 '(106823) ЛОкситан Шампунь Сила и Густота 300мл 476',
 '(192300) PL Перчатки латексные нестерильные р.M №10 675',
 '(67740) Аторвастатин таб.п.п.о.20мг №30 685',
 '(66406) Ополаскиватель д/полости рта Лакалют Уайт 300мл -1',
 '(35332) СТИКС Крем "Авокадо" 50 мл, уп №1 (150) 498',
 '(3434) Чай "Волшебная сила" [фильтр-пакеты 2г] N20 -1',
 '(183248) Катетер-баллон "Фолея" 2-х ходовой (№18) №1 761']